# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather = "../output_data/weather_data.csv"
weather_data = pd.read_csv(weather)
weather_data

,Unnamed: 0,City,Temperatures (F),Latitude,Longitude,Humidity %,Wind (mph),Cloudiness
0,0,kruisfontein,62.01,-34.0033,24.7314,86,1.01,54
1,1,khatanga,-27.20,71.9667,102.5000,80,12.59,13
2,2,new norfolk,58.37,-42.7826,147.0587,64,1.99,100
3,3,rampura,57.15,30.2500,75.2333,49,1.97,0
4,4,tullamore,34.50,53.2739,-7.4889,69,7.00,28
...,...,...,...,...,...,...,...,...
562,562,norfolk,27.07,42.1668,-71.2495,86,9.22,96
563,563,veriora,14.00,58.0079,27.3551,85,5.75,90
564,564,luganville,85.95,-15.5333,167.1667,54,4.54,4
565,565,llanes,53.01,43.4198,-4.7549,68,5.01,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [79]:
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['Humidity %']
max_humidity = humidity.max()

100

In [22]:
fig = gmaps.figure(zoom_level = 2, center = (0,0))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
ideal_weather = weather_data.loc[(weather_data['Temperatures (F)']<80) &
                                 (weather_data['Temperatures (F)']> 65)& 
                                 (weather_data['Humidity %']< 50) &
                                (weather_data['Cloudiness']< 60)]
ideal_weather

,Unnamed: 0,City,Temperatures (F),Latitude,Longitude,Humidity %,Wind (mph),Cloudiness
132,132,port augusta,75.20,-32.5000,137.7667,46,10.36,0
228,228,jega,71.13,12.2175,4.3792,18,8.88,7
260,260,luderitz,68.05,-26.6481,15.1594,47,24.92,0
333,333,makakilo city,75.99,21.3469,-158.0858,47,6.91,1
397,397,broken hill,77.00,-31.9500,141.4333,31,12.66,0
436,436,brawley,68.00,32.9787,-115.5303,26,3.44,1
437,437,ouallam,74.01,14.3188,2.0866,19,10.63,0
516,516,ewa beach,76.17,21.3156,-158.0072,47,6.91,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
hotel_df = ideal_weather.filter(['City'])
hotel_df['Hotel Name'] = ""
hotel_df['Lat'] = ""
hotel_df['Lng']= ""
hotel_df

,City,Hotel Name,Lat,Lng
132,port augusta,,,
228,jega,,,
260,luderitz,,,
333,makakilo city,,,
397,broken hill,,,
436,brawley,,,
437,ouallam,,,
516,ewa beach,,,


In [75]:
for index, row in ideal_weather.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    coords=f'{latitude},{longitude}'
    try:
        params={ 
            "location":coords,
            "radius" : 5000,
            "type" :"lodging", 
            "key" : g_key
            }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response=requests.get(base_url, params=params).json()
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = response['results'][0]['geometry']['location']["lat"]
        hotel_df.loc[index,'Lng'] = response['results'][0]['geometry']['location']["lng"]
    except:
        print(f'No nearby hotels to {city}.')
        hotel_df.drop(index)
        

No nearby hotels to ouallam.


In [76]:
hotel_df

,City,Hotel Name,Lat,Lng
132,port augusta,Majestic Oasis Apartments,-32.4889,137.763
228,jega,ALH. Nasiru Ibrahim,12.2229,4.3755
260,luderitz,LÜDERITZ NEST HOTEL,-26.653,15.1496
333,makakilo city,Marriott's Ko Olina Beach Club,21.3316,-158.12
397,broken hill,Royal Exchange Hotel,-31.9578,141.466
436,brawley,Brawley Inn Hotel & Conference Center,32.9789,-115.551
437,ouallam,,,
516,ewa beach,Base Lodging,21.3345,-157.966


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotels= hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [88]:
# Add marker layer ontop of heat map
hotel_loc = []
for index, row in hotel_df.iterrows():
    hotel= [f'{row["Lat"]}, {row["Lng"]}']
    hotel_loc.append(hotel)
markers = gmaps.marker_layer(hotel_loc)

fig.add_layer(markers)
# Display figure
fig


ValueError: not enough values to unpack (expected 2, got 1)